## Let's import the needed libraries for our notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import glob
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import os
import pathlib
import PIL
import PIL.Image
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds

from IPython.display import clear_output
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix
from tensorflow.keras import callbacks
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

![](https://i.ibb.co/7YqhnXg/wallpaper.png)

Yes I took this photo using my lightbox and Sony a6400 + Sigma 18-50 f2.8

## Let's declare some variables

In [ ]:
batch_size = 8
img_height = 1080
img_width = 1620
resized_height = 384
resized_width = 384
epochs=150

## Let's get all the images and let's get a count

In [ ]:
training_dir = '/kaggle/input/nespresso-capsules-dataset/'
training_dir = pathlib.Path(training_dir)
print(len(list(training_dir.glob('*/*.JPG'))))

## Let's create the training dataset which will be 80% of the total # of images

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  training_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

## As we are using the .repeat() keyword below in the train_ds, we need to get the steps per epoch or loop infinitely

In [ ]:
steps_per_epoch = len(train_ds)

## Then let's use the remaining 20% as the validation/ test dataset

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  training_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

## Let's look at the classes

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Let's show a random actual image per class

In [ ]:
plt.figure(figsize=(16, 16))
    
for i in range(len(class_names)):
    filtered_ds = train_ds.filter(lambda x, l: tf.math.equal(l[0], i))
    for image, label in filtered_ds.take(1):
        ax = plt.subplot(4, 4, i+1)
        plt.imshow(image[0].numpy().astype('uint8'))
        plt.title(class_names[label.numpy()[0]])
        plt.axis('off')    

## Let's use AUTOTUNE to help dynamically configure and tweak optimal resource allocation during runtime https://stackoverflow.com/questions/56613155/tensorflow-tf-data-autotune

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).repeat().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Let's create the augmentation layer that we will plug into our model's sequential layer

In [ ]:
data_augmentation_layer = tf.keras.Sequential([      
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomZoom(0.15),
    tf.keras.layers.RandomTranslation(height_factor=(-0.05, 0.05), width_factor=(-0.05, 0.05)),
    tf.keras.layers.RandomFlip('horizontal_and_vertical',input_shape=(img_width, img_height, 3)),
    tf.keras.layers.CenterCrop(height=img_height,width=img_height), #knowing that the image is 1080 at the shortest, we crop from the center to get a 1080 x 1080 image
    tf.keras.layers.Resizing(height=resized_height,width=resized_width), #after cropping 1080 x 1080 from the center, we now resize down to our intended image that will be passed to the model
], name='data_augmentation')

## Let's take one of images and see the augmentations being applied to the image

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation_layer(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[1].numpy().astype("uint8"))
        plt.axis("off")

## Let's define our sequential model

In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    data_augmentation_layer, 
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),    
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),    
    layers.Dropout(0.2),
    layers.Dense(512, activation='relu'),  
    layers.BatchNormalization(),
    layers.Dense(num_classes, activation='softmax'),  
])

## Let's compile the model and declare the optimizer, loss, and metrics that the model would use

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])

## Let's use an early stopping callback to combat overfitting

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='loss', mode='auto',min_delta=0.0005, verbose=1, patience=10, restore_best_weights=True)

## Let's start training the model

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    steps_per_epoch = steps_per_epoch,
    verbose=1,
    callbacks = [early_stopping]
)

## Let's look at the model's architecture

In [ ]:
model.summary()

## Let's get the accuracy and loss of the training process

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

## Let's graph the accuracy and loss visually check if it's converging

In [ ]:
epochs_range = range(len(acc))

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 2)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.subplot(1, 2, 1)
plt.plot(epochs_range, loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')
plt.show()

## Let's plot the model which shows more info compared to the model.summary

In [ ]:
plot_model(model,show_shapes=True, show_layer_names=True)

## Let's try to look at some of the images in the validation dataset

In [ ]:
plt.figure(figsize=(12, 12))
for images, labels in val_ds.take(1):
    for i in range(batch_size):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Let's evaluate the model using the validation dataset

In [ ]:
score = model.evaluate(val_ds, verbose=0)

## Let's print the loss and accuracy

In [ ]:
print("Accuracy: {}%, Loss:{}".format(score[1]*100, score[0]))

## Let's do the prediction manually using the validation dataset

In [ ]:
prediction_labels = []
true_labels = []

for images, labels in val_ds.take(-1):  
    preds = model.predict(images.numpy(), verbose=0).round()
    prediction_labels.append(np.argmax(preds, axis = - 1))
    true_labels.append(labels.numpy())

## Let's get the true and prediction labels and flatten the predictions and true labels which we will visualize with a confusion matrix

In [ ]:
true_labels = np.concatenate(true_labels).tolist()
prediction_labels = np.concatenate(prediction_labels).tolist()

## Let's plot the confusion matrix

In [ ]:
sns.set(rc={'figure.figsize':(8,6)})

ax = sns.heatmap(confusion_matrix(true_labels, prediction_labels), annot=True, cmap='Blues', fmt='g')
ax.set_title('Seaborn Confusion Matrix with labels');
ax.set_xlabel('Predicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(class_names)
ax.yaxis.set_ticklabels(class_names)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

plt.show()

## Let's visualize some of the correct predictions

In [ ]:
plt.figure(figsize=(16, 16))
incorrect_images = 0

for images, labels in val_ds.take(-1):
    for i in range(len(images)):
        predicted_label = np.argmax(np.round(model.predict(tf.expand_dims(images[i], 0)))) #this is needed as the model predicts a batch of iamges
        true_label = labels[i]
        
        if((predicted_label ==true_label) & (incorrect_images <=8)):
            incorrect_images = incorrect_images + 1
            ax = plt.subplot(3, 3, incorrect_images)         
            plt.axis("off")
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title("True: {} \nPredicted: {}".format(class_names[int(true_label)],class_names[int(predicted_label)]))
            
clear_output()

## Let's visualize some of the incorrectly predicted portraits

In [ ]:
plt.figure(figsize=(16, 16))

incorrect_images = 0

for images, labels in val_ds.take(-1):
    for i in range(len(images)):
        predicted_label = np.argmax(np.round(model.predict(tf.expand_dims(images[i], 0)))) #this is needed as the model predicts a batch of iamges
        true_label = labels[i]
        
        if((predicted_label !=true_label) & (incorrect_images <=8)):
            incorrect_images = incorrect_images + 1
            ax = plt.subplot(3, 3, incorrect_images)
            plt.axis("off")
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title("True: {} \nPredicted: {}".format(class_names[int(true_label)],class_names[int(predicted_label)]))
            
clear_output()

## Take aways
We created a basic image classifying neural network with decent performance, but I'm sure this can be improved with further hyperparameterization and image augmentation techniques, even a newer architecture. 

I hope that my notebook and my dataset have taught you a thing or two on how to create a multi-class image classification model with Tensorflow and Keras.

Looking forward to your notebooks and your solutions! Im excited to learn more techniques on Image Classification using Deep Learning!

Happy Kaggling!